In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import timedelta
import matplotlib.pyplot as plt
%matplotlib inline

parameters = {
    "paths": {
        "meteo": "../data/barcelona_meteo.csv",
        "demand": "../data/CE_EO_Demanda.xlsx"
    }
}

In [ ]:
demand_df = pd.read_excel(parameters["paths"]["demand"], sheet_name="Demanda")
print(f"Number of rows: {demand_df.shape[0]}")
demand_df.head(5)

In [ ]:
#Prints the NaN Values per Column of a dataset
def checkNaNValues(df):
    return print(f"NaN Count per Column", df.isna().any(axis=1).sum())
checkNaNValues(demand_df)
demand_df["Date"] = pd.to_datetime(demand_df["Date"])



In [ ]:
#Check Missing Time Data
def checkMissingData(df, date_column):
    min_date = df[date_column].min()
    max_date = df[date_column].max()

    date_range = pd.date_range(start=min_date, end=max_date, freq='15T')

    missing_dates = date_range[~date_range.isin(df[date_column])]
    print(f"Number of Missing Values: {missing_dates.shape[0]}")
    return None if missing_dates.shape[0] == 0 else missing_dates
missing_data = checkMissingData(demand_df, "Date")
#No Missing Values in demand_df

In [ ]:
meteo_df = pd.read_csv(parameters["paths"]["meteo"])
print(f"Number of rows: {demand_df.shape[0]}")
meteo_df["date"] = pd.to_datetime(meteo_df["date"], format='%m/%d/%y %H:%M')
meteo_df.head(5)


In [ ]:
checkNaNValues(meteo_df)
meteo_df.set_index('date', inplace=True)
meteo_resampled_df = meteo_df.resample('15T').asfreq()
meteo_resampled_df.reset_index(inplace=True)

In [ ]:
meteo_resampled_df.head(10)
missing_meteo_data = checkMissingData(meteo_resampled_df, "date")

In [ ]:
#Data Interpolation
def interpolateData(df, columns, method):
    interpolated_df = df.copy()
    for col in columns:
        interpolated_df[col] = interpolated_df[col].interpolate(method=method)
    return interpolated_df
columns2interpolate = meteo_resampled_df.columns.drop("date")
print(meteo_resampled_df.head(10))
df_interpolated = interpolateData(meteo_resampled_df, columns2interpolate, 'linear')
print("------------------Interpolation------------------")
print("Hola")
df_interpolated.head(10)


In [ ]:
#Outliers 
def getNumberOfOutliersPerColumn(df, columns_list):
    for col in columns_list:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1

        #Calcular los límites inferiores y superiores para identificar outliers
        lim_inf = q1 - 1.5 * iqr
        lim_sup = q3 + 1.5 * iqr

        outliers = ((df[col] < lim_inf) | (df[col] > lim_sup))
        #print(outliers.index[outliers])
        print(f"For {col} Column, the Number of Outliers is {outliers.sum()}\n")

getNumberOfOutliersPerColumn(df_interpolated, columns2interpolate)

In [ ]:
start_date = df_interpolated['date'].min()
end_date = start_date + timedelta(days=7)
df_week1 = df_interpolated.loc[(df_interpolated['date'] >= start_date) & (df_interpolated['date'] <= end_date)]

plt.figure(figsize=(12, 6))
plt.plot(df_week1['Temperatura'], label='Duración Radiación')
plt.title('Temperatura Semana 1')
plt.xlabel('Delta 15 mins')
plt.ylabel('Grados Cº')
plt.legend()

plt.show()

In [ ]:
getNumberOfOutliersPerColumn(df_interpolated, columns2interpolate)